In [1]:
import sqlite3
import time
import statistics

In [2]:
db_path = 'Dirigo.sqlite'

query = '''
 WITH IdentifiedTruck AS (
  SELECT Object_id AS TruckID
  FROM OCEL_Truck
  WHERE LPT = '926VFT'
)
  SELECT *
  FROM OCEL_O2O
  WHERE Source_object_id = (SELECT TruckID FROM IdentifiedTruck)
    AND Target_object_id = 'Pcp18';
'''
repeat = 1000

conn = sqlite3.connect(db_path)
cursor = conn.cursor()


durations = []

for i in range(repeat):
    start_time = time.time()
    cursor.execute(query)
    cursor.fetchall()  
    end_time = time.time()
    
    duration = (end_time - start_time) * 1000  
    durations.append(duration)
    # print(f"Run {i+1}: {duration:.4f} ms")


avg = statistics.mean(durations)
std_dev = statistics.stdev(durations)

print("\n=== Execution Statistics (Dirigo) ===")
print(f"Average execution time: {avg:.4f} ms")
print(f"Standard deviation: {std_dev:.4f} ms")


cursor.close()
conn.close()


=== Execution Statistics (Dirigo) ===
Average execution time: 0.0268 ms
Standard deviation: 0.0178 ms


In [3]:
db_path = 'ACEL.sqlite'

query = """
WITH IdentifiedTruck AS (
  SELECT ObjectId AS TruckID
  FROM ACEL_Objects
  WHERE LPT = '926VFT'
),
TruckEvents AS (
SELECT e.*
FROM ACEL_Events e
WHERE EXISTS (
  SELECT 1
  FROM json_each(REPLACE(e.Objects, '''', '"'))
  WHERE json_each.value = (SELECT TruckID FROM IdentifiedTruck))
 ),
RelationID As (
  SELECT 
    rel.value AS RelationId
  FROM TruckEvents e,
       json_each(REPLACE(e.ObjectChanges, '''', '"')) obj_change,
       json_each(REPLACE(e.Relations, '''', '"')) rel
  WHERE json_extract(obj_change.value, '$.Attribute') = 'Pickup Plan ID'
  AND json_extract(obj_change.value, '$.NewValue') = 'Pcp18' 
),
Qualifiers AS (
SELECT *
FROM ACEL_Relations 
WHERE RelationId = (SELECT RelationId FROM RelationID)
),
SELECTEDTruckEvents AS(
SELECT * 
FROM TruckEvents, json_each(REPLACE(Relations, '''', '"')) AS jc
WHERE jc.value = (SELECT RelationId FROM RelationID)
)
SELECT 
	r.Source,
  CASE 
      WHEN json_extract(rc.value, '$.ChangeStatus') = 'addedTarget' THEN 'is assigned to'
      WHEN json_extract(rc.value, '$.ChangeStatus') = 'deletedTarget' THEN 'is dropped from'
  END AS Qualifier,
  e.Timestamp,
	json_extract(rc.value, '$.Target') AS 'Target'
FROM 
    SELECTEDTruckEvents e,
    json_each(REPLACE(e.RelationChanges, '''', '"')) AS rc
JOIN 
    Qualifiers r
    ON r.RelationId = json_extract(rc.value, '$.RelationId')
    AND r.Type = CASE 
                    WHEN json_extract(rc.value, '$.ChangeStatus') = 'addedTarget' THEN 'is assigned to'
                    WHEN json_extract(rc.value, '$.ChangeStatus') = 'deletedTarget' THEN 'is dropped from'
                END
WHERE 
    json_extract(rc.value, '$.RelationId') IN (
        SELECT RelationId FROM SELECTEDTruckEvents
    );
"""

repeat = 1000

conn = sqlite3.connect(db_path)
cursor = conn.cursor()


durations = []

for i in range(repeat):
    start_time = time.time()
    cursor.execute(query)
    cursor.fetchall()  
    end_time = time.time()
    
    duration = (end_time - start_time) * 1000  
    durations.append(duration)
    # print(f"Run {i+1}: {duration:.4f} ms")


avg = statistics.mean(durations)
std_dev = statistics.stdev(durations)

print("\n=== Execution Statistics (ACEL) ===")
print(f"Average execution time: {avg:.4f} ms")
print(f"Standard deviation: {std_dev:.4f} ms")


cursor.close()
conn.close()


=== Execution Statistics (ACEL) ===
Average execution time: 1.9066 ms
Standard deviation: 0.0889 ms
